# Feature Engineering

## Cargue de librerias y parametros

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Agrega la ruta del directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Ahora puedes importar tus módulos
from procesamiento_datos import *

Función de procesamiento de datos cargadas correctamente.


In [2]:
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
datos_raw = "..//data/raw/"
datos_processed = "..//data/processed/"

## Cargue de los dataframes

In [23]:

# Cargar los datos
df_facturas = cargar_datos(datos_raw + "Historico_Facturas.csv")
df_cierres = cargar_datos(datos_raw + "Historico_Cierres.csv")
df_negocios = cargar_datos(datos_raw + "Tipos_Negocio.csv")
df_ipc = cargar_datos(datos_raw + "Historico_IPC.csv", sep=';')
df_tasas = cargar_datos(datos_raw + "Historico_Tasas_Credito.csv", sep=';')
df_pib = cargar_datos(datos_raw + "Historico_PIB_Corriente_Desc.csv")
df_sml = cargar_datos(datos_raw + "Historico_Salario_Minimo.csv")
df_clientes = cargar_datos(datos_raw + "Clientes_EEFF.csv")

# Pruebas

In [25]:
convertir_fecha(df_facturas,['Fecha_expedicion','Fecha_pago','Fecha_vencimiento'])
df_facturas

,NIT,id_factura,Fecha_expedicion,Estado,Total factura,Saldo,Fecha_pago,Fecha_vencimiento,Valor_pagado
0,42ecd8cb62d29625ad81914f17a48c219acb5d2be2daf0...,186537,2025-06-18,CONFIRMADA,1035398.0,1035398.0,NaT,2025-07-18,0.0
1,46f7f7e4cdea7bc958d0999864222c63474cbd4f9751be...,177214,2024-08-28,PAGADA,773500.0,0.0,2024-09-25,2024-09-27,773500.0
2,727d72342d46d4357539ec474dae79ce388234624a74d7...,161840,2022-11-25,PAGADA,11900000.0,0.0,2024-11-28,2022-12-25,11900000.0
3,c1d637e1f5632d028983dbc5a46b8ea1dcae5a7aee1331...,161841,2022-11-25,PAGADA,11900000.0,0.0,2024-08-07,2022-12-25,11900000.0
4,bec0e3be4b4d494a8b98d121499d2ff4d8bae3cdb5526b...,177216,2024-08-28,PAGADA,773500.0,0.0,2025-01-10,2024-09-27,773500.0
...,...,...,...,...,...,...,...,...,...
61388,97b08d62d4b078cad69cc9a52a325ddf8b88078a21ef0d...,181279,2025-02-13,PAGADA,41562106.0,0.0,2025-06-09,2025-03-15,41562106.0
61389,97b08d62d4b078cad69cc9a52a325ddf8b88078a21ef0d...,184044,2025-03-06,PAGADA,43257302.0,0.0,2025-03-20,2025-04-05,43257302.0
61390,97b08d62d4b078cad69cc9a52a325ddf8b88078a21ef0d...,185064,2025-04-14,PAGADA,45900437.0,0.0,2025-04-16,2025-05-14,45900437.0
61391,97b08d62d4b078cad69cc9a52a325ddf8b88078a21ef0d...,185619,2025-05-09,PAGADA,45269411.0,0.0,2025-06-23,2025-06-08,45269411.0


In [26]:
fecha_referencia = pd.to_datetime('2025-06-30')

def calcular_dias_demora(row):
    if row ['Estado'] == 'PAGADA' and row ['Fecha_pago'] != 'NaT':
        return ( row['Fecha_pago'] - row['Fecha_vencimiento']).days
    else:
        return ( fecha_referencia - row['Fecha_vencimiento']).days

df_facturas ['dias_demora'] = df_facturas.apply(calcular_dias_demora, axis=1)

In [30]:
#si los días de mora son negativos o iguales a 0 el habito de pago es bueno = 1 , si son mayores a 0 el habito de pago es malo = 0 
df_facturas['habito_pago'] = df_facturas['dias_demora'].apply(lambda x: 1 if x <= 30 else 0)
df_facturas['ratio_pago'] =df_facturas['Valor_pagado']/df_facturas['Total factura']

In [31]:
df_facturas_agg = df_facturas.copy()
df_facturas_agg = df_facturas_agg.groupby(['NIT']).agg(
    Total_facturado=('Total factura','sum'),
    Total_por_pagar = ('Saldo','sum'),
    Total_pagado = ('Valor_pagado','sum'),
    Cantidad_facturas= ('id_factura','count'),
    Dias_mora_promedio=('dias_demora','mean'),
    porcentaje_pagos_tiempo = ('habito_pago','mean'),
    promedio_ratio_pago = ('ratio_pago','mean'),
    desv_dias_demora = ('dias_demora','std'),
    maximo_dias_demora = ('dias_demora','max'),
    facturas_vencidas = ('Estado',lambda x: (x !="PAGADA").sum())
).reset_index()
df_facturas_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2697 entries, 0 to 2696
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NIT                      2697 non-null   object 
 1   Total_facturado          2697 non-null   float64
 2   Total_por_pagar          2697 non-null   float64
 3   Total_pagado             2697 non-null   float64
 4   Cantidad_facturas        2697 non-null   int64  
 5   Dias_mora_promedio       2697 non-null   float64
 6   porcentaje_pagos_tiempo  2697 non-null   float64
 7   promedio_ratio_pago      2697 non-null   float64
 8   desv_dias_demora         1208 non-null   float64
 9   maximo_dias_demora       2697 non-null   int64  
 10  facturas_vencidas        2697 non-null   int64  
dtypes: float64(7), int64(3), object(1)
memory usage: 231.9+ KB


In [32]:
df_facturas_agg['Factura_promedio'] = round(df_facturas_agg['Total_facturado']/df_facturas_agg['Cantidad_facturas'])
df_facturas_agg.head()

,NIT,Total_facturado,Total_por_pagar,Total_pagado,Cantidad_facturas,Dias_mora_promedio,porcentaje_pagos_tiempo,promedio_ratio_pago,desv_dias_demora,maximo_dias_demora,facturas_vencidas,Factura_promedio
0,00019431b6126f9b9bb9bae1c0b02635843eb251e1c887...,2998928.0,0.0,2998928.0,7,1190.857143,0.000000,1.0,635.689502,1973,0,428418.0
1,0018bd0b9cac8d76713b97535738e3004c44cc80821907...,113640820.0,0.0,113640820.0,55,887.200000,0.018182,1.0,567.265416,2272,0,2066197.0
2,00590f245f22ca6dc9fe272c2e79747bd8da43ff1ab6e3...,783439.0,0.0,783439.0,2,990.000000,0.000000,1.0,558.614357,1385,0,391720.0
3,007025963bf2a2d6fa88441ff1badad81eaef22e2250b1...,90224784.0,0.0,90224784.0,46,994.934783,0.043478,1.0,652.313265,2377,0,1961408.0
4,0074eeb2e1a555937c7c58c80149c49411855b1b8d7210...,773500.0,0.0,773500.0,1,176.000000,0.000000,1.0,NaN,176,0,773500.0


## Creación de features

In [22]:
# Crear sistema de scoring
def calcular_score_credito(row):
    score = 0
    
    # 1. Porcentaje de pagos a tiempo (max 30 puntos)
    score += min(row['porcentaje_pagos_tiempo'] * 0.3, 30)
    
    # 2. Días de mora promedio (max 25 puntos)
    if row['promedio_dias_mora'] <= 0:
        score += 25
    elif row['promedio_dias_mora'] <= 30:
        score += 15
    elif row['promedio_dias_mora'] <= 60:
        score += 5
    
    # 3. Ratio de pago promedio (max 20 puntos)
    score += min(row['ratio_pago_promedio'] * 20, 20)
    
    # 4. Facturas vencidas actuales (max 15 puntos)
    if row['facturas_vencidas_actuales'] == 0:
        score += 15
    elif row['facturas_vencidas_actuales'] <= 2:
        score += 7
    
    # 5. Volatilidad en pagos (max 10 puntos)
    if pd.isna(row['volatilidad_dias_mora']) or row['volatilidad_dias_mora'] <= 15:
        score += 10
    elif row['volatilidad_dias_mora'] <= 30:
        score += 5
    
    return score

# Calcular el score para cada cliente
indicadores_cliente['score_credito'] = indicadores_cliente.apply(calcular_score_credito, axis=1)

# Establecer categorías de riesgo basadas en el score
def categorizar_riesgo(score):
    if score >= 85:
        return 'A - Riesgo Bajo'
    elif score >= 70:
        return 'B - Riesgo Moderado Bajo'
    elif score >= 50:
        return 'C - Riesgo Moderado'
    elif score >= 30:
        return 'D - Riesgo Moderado Alto'
    else:
        return 'E - Riesgo Alto'

indicadores_cliente['categoria_riesgo'] = indicadores_cliente['score_credito'].apply(categorizar_riesgo)

# Mostrar distribución de scores y categorías
print("\nDistribución de Scores de Crédito:")
print(indicadores_cliente['score_credito'].describe())

print("\nDistribución de Categorías de Riesgo:")
print(indicadores_cliente['categoria_riesgo'].value_counts())


Distribución de Scores de Crédito:
count    2927.000000
mean       30.229713
std        13.017618
min         6.875000
25%        21.723810
50%        30.000000
75%        30.000000
max        87.000000
Name: score_credito, dtype: float64

Distribución de Categorías de Riesgo:
categoria_riesgo
D - Riesgo Moderado Alto    1741
E - Riesgo Alto             1038
A - Riesgo Bajo              122
C - Riesgo Moderado           18
B - Riesgo Moderado Bajo       8
Name: count, dtype: int64


In [23]:
# Resetear el índice para tener id_cliente como columna
indicadores_cliente = indicadores_cliente.reset_index()

# Realizar el merge con df_prueba
df_final = df_prueba.merge(indicadores_cliente, 
                          left_on='id_cliente',
                          right_on='id_cliente',
                          how='left')

print("\nColumnas en el DataFrame final:")
print(df_final.columns.tolist())

print("\nDimensiones del DataFrame final:")
print(f"Filas: {df_final.shape[0]}, Columnas: {df_final.shape[1]}")

# Verificar si hay clientes sin score
clientes_sin_score = df_final[df_final['score_credito'].isna()]['id_cliente'].nunique()
print(f"\nClientes sin score de crédito: {clientes_sin_score}")

# Mostrar las primeras filas del DataFrame final con las columnas más relevantes
columnas_relevantes = ['id_cliente', 'fecha_cierre', 'cuenta_cobrar', 'dias_mora', 
                      'calificacion', 'score_credito', 'categoria_riesgo', 
                      'promedio_dias_mora', 'porcentaje_pagos_tiempo']

print("\nMuestra del DataFrame final con columnas relevantes:")
print(df_final[columnas_relevantes].head())


Columnas en el DataFrame final:
['fecha_cierre', 'id_cliente', 'dias_mora', 'calificacion', 'cuenta_cobrar', 'deterioro', 'id_negocio', 'cod_negocio', 'id_linea', 'nombre_linea', 'codigo_camara', 'codigo_clase_identificacion', 'clase_identificacion', 'cod_ciiu_act_econ_pri', 'categoria_matricula', 'id_hash', 'presente_en_facturas', 'presente_en_cierres', 'promedio_dias_mora', 'porcentaje_pagos_tiempo', 'total_facturas', 'monto_promedio_facturas', 'ratio_pago_promedio', 'volatilidad_dias_mora', 'maximo_dias_mora', 'facturas_vencidas_actuales', 'saldo_pendiente_total', 'score_credito', 'categoria_riesgo']

Dimensiones del DataFrame final:
Filas: 15916, Columnas: 29

Clientes sin score de crédito: 36

Muestra del DataFrame final con columnas relevantes:
                                          id_cliente fecha_cierre  \
0  bda703a41ee55dcb911f17164009bc87636552bd5a641b...   2023-01-31   
1  bda703a41ee55dcb911f17164009bc87636552bd5a641b...   2023-01-31   
2  a6fbe7c3a37fa63f96d9ac3efe82

## Incorporación de variables macroeconómicas
Agregaremos las variables macroeconómicas (IPC, PIB, tasas de crédito y salario mínimo) al DataFrame final. Estas variables son importantes para capturar el contexto económico en el que se desarrollan los comportamientos de pago.

In [24]:
def clean_data(df_sml):
    # Cambiar el nombre de la columna 'Serie' a 'Fecha'
    df_sml = df_sml.rename(columns={'Serie': 'Fecha'})
    # Cambiar el nombre de la columna 'Salario\xa0mínimo\xa0mensual' a 'SML'
    df_sml = df_sml.rename(columns={'Salario\xa0mínimo\xa0mensual': 'SML'})
    # Eliminar las columnas: 'Auxilio de transporte mensual', 'Salario mínimo mensual, sector rural', 'Salario mínimo mensual, sector urbano alto'
    df_sml = df_sml.drop(columns=['Auxilio\xa0de\xa0transporte\xa0mensual', 'Salario\xa0mínimo\xa0mensual,\xa0sector\xa0rural', 'Salario\xa0mínimo\xa0mensual,\xa0sector\xa0urbano\xa0alto'])
    return df_sml

df_sml = clean_data(df_sml.copy())
df_sml.head()

,Fecha,SML
0,1950-01-31,.
1,1950-02-28,.
2,1950-03-31,.
3,1950-04-30,.
4,1950-05-31,.


In [25]:
def clean_data(df_ipc):
    # Eliminar las columnas: 'Inflación sin alimentos ni regulados, anual', 'Inflación sin alimentos, anual'
    df_ipc = df_ipc.drop(columns=['Inflación\xa0sin\xa0alimentos\xa0ni\xa0regulados,\xa0anual', 'Inflación\xa0sin\xa0alimentos,\xa0anual'])
    # Cambiar el nombre de la columna 'Inflación\xa0núcleo\xa015 anual' a 'IPC'
    df_ipc = df_ipc.rename(columns={'Inflación\xa0núcleo\xa015 anual': 'IPC'})
    return df_ipc

df_ipc = clean_data(df_ipc.copy())
df_ipc.head()

,Fecha,IPC
0,31/01/1999,"16,47"
1,28/02/1999,"15,35"
2,31/03/1999,"13,92"
3,30/04/1999,"12,98"
4,31/05/1999,"12,12"


In [26]:
def clean_data(df_pib):
    # Eliminar las columnas: '1.01. Valor agregado bruto', '1.01.01. Agricultura, ganadería, caza, silvicultura y pesca' y 13 otras columnas
    df_pib = df_pib.drop(columns=['1.01. Valor agregado bruto', '1.01.01. Agricultura, ganadería, caza, silvicultura y pesca', '1.01.02. Explotación de minas y canteras', '1.01.03. Industrias manufactureras', '1.01.04. Suministro de electricidad, gas, vapor y\naire acondicionado (*)', '1.01.05. Construcción', '1.01.06. Comercio al por mayor y al por menor\n(*)', '1.01.07. Información y comunicaciones', '1.01.08. Actividades financieras y de seguros', '1.01.09. Actividades inmobiliarias', '1.01.10. Actividades profesionales, científicas y\ntécnicas (*)', '1.01.11. Administración pública, defensa,\neducación y salud (*)', '1.01.12. Actividades artísticas, de\nentretenimiento y recreación y otras actividades\nde servicios (*)', '1.02. Impuestos menos subvenciones sobre los\nproductos', '\xa0\nLos\xa0valores\xa0ausentes\xa0se\xa0indican\xa0con\xa0un\xa0punto\xa0(.)\nDescargado\xa0del\xa0sistema\xa0del\xa0Banco\xa0de\xa0la\xa0República:\xa008/08/2025\xa0\xa06:22:02\xa0PM'])
    # Cambiar el nombre de la columna '1. PIB reportado' a 'PIB'
    df_pib = df_pib.rename(columns={'1. PIB reportado': 'PIB'})
    return df_pib

df_pib = clean_data(df_pib.copy())
df_pib.head()

,Fecha,PIB
0,2005-01-31,"81365,11"
1,2005-02-28,"81365,11"
2,2005-03-31,"81365,11"
3,2005-04-30,"84193,31"
4,2005-05-31,"84193,31"


In [27]:
def clean_data(df_tasas):
    # Eliminar las columnas: 'Créditos de tesorería', 'Créditos ordinarios' y 4 otras columnas
    df_tasas = df_tasas.drop(columns=['Créditos\xa0de\xa0tesorería', 'Créditos\xa0ordinarios', 'Créditos\xa0preferenciales', 'Tasas\xa0de\xa0Colocación\xa0Banco\xa0de\xa0la\xa0República', 'Tasas\xa0de\xa0Colocación\xa0sin\xa0tesoreria', 'Tasas\xa0de\xa0Colocación\xa0total'])
    # Cambiar el nombre de la columna 'Créditos\xa0de\xa0consumo' a 'Tasa'
    df_tasas = df_tasas.rename(columns={'Créditos\xa0de\xa0consumo': 'Tasa'})
    return df_tasas

df_tasas = clean_data(df_tasas.copy())
df_tasas.head()

,Fecha,Tasa
0,31/03/1998,"42,24"
1,30/04/1998,"44,08"
2,31/05/1998,"45,32"
3,30/06/1998,"48,41"
4,31/07/1998,"51,57"


In [ ]:
# Preparar datos macroeconómicos
# IPC
df_ipc['Fecha'] = pd.to_datetime(df_ipc['Fecha'])
df_ipc = df_ipc.rename(columns={'Fecha': 'fecha_cierre', 'IPC': 'ipc'})
df_ipc['ipc'] = pd.to_numeric(df_ipc['ipc'], errors='coerce')

# PIB
df_pib['Fecha'] = pd.to_datetime(df_pib['Fecha'])
df_pib = df_pib.rename(columns={'Fecha': 'fecha_cierre', 'PIB': 'pib'})
df_pib['pib'] = pd.to_numeric(df_pib['pib'], errors='coerce')

# Tasas de crédito
df_tasas['Fecha'] = pd.to_datetime(df_tasas['Fecha'])
df_tasas = df_tasas.rename(columns={'Fecha': 'fecha_cierre', 'Tasa': 'tasa_credito'})
df_tasas['tasa_credito'] = pd.to_numeric(df_tasas['tasa_credito'], errors='coerce')

# Salario mínimo
df_sml['Fecha'] = pd.to_datetime(df_sml['Fecha'])
df_sml = df_sml.rename(columns={'Fecha': 'fecha_cierre', 'SML': 'salario_minimo'})
df_sml['salario_minimo'] = pd.to_numeric(df_sml['salario_minimo'], errors='coerce')

# Merge de todas las variables macroeconómicas con df_final
df_final = df_final.merge(df_ipc[['fecha_cierre', 'ipc']], on='fecha_cierre', how='left')
df_final = df_final.merge(df_pib[['fecha_cierre', 'pib']], on='fecha_cierre', how='left')
df_final = df_final.merge(df_tasas[['fecha_cierre', 'tasa_credito']], on='fecha_cierre', how='left')
df_final = df_final.merge(df_sml[['fecha_cierre', 'salario_minimo']], on='fecha_cierre', how='left')

In [34]:
# Verificar los tipos de datos de las columnas macroeconómicas
print("Tipos de datos de las columnas macroeconómicas:")
print("\nTipos de datos en df_ipc:")
print(df_ipc.dtypes)
print("\nPrimeras filas de df_ipc:")
print(df_ipc.head())

print("\nTipos de datos en df_final después del merge:")
print(df_final[['ipc', 'pib', 'tasa_credito', 'salario_minimo']].dtypes)
print("\nPrimeras filas de las variables macro en df_final:")
print(df_final[['ipc', 'pib', 'tasa_credito', 'salario_minimo']].head())

Tipos de datos de las columnas macroeconómicas:

Tipos de datos en df_ipc:
fecha_cierre    datetime64[ns]
ipc                     object
dtype: object

Primeras filas de df_ipc:
  fecha_cierre    ipc
0   1999-01-31  16,47
1   1999-02-28  15,35
2   1999-03-31  13,92
3   1999-04-30  12,98
4   1999-05-31  12,12

Tipos de datos en df_final después del merge:
ipc               object
pib               object
tasa_credito      object
salario_minimo    object
dtype: object

Primeras filas de las variables macro en df_final:
     ipc        pib tasa_credito salario_minimo
0  11,84  394716,61        30,84   1.160.000,00
1  11,84  394716,61        30,84   1.160.000,00
2  11,84  394716,61        30,84   1.160.000,00
3  11,84  394716,61        30,84   1.160.000,00
4  11,84  394716,61        30,84   1.160.000,00


In [42]:
# Calcular variaciones mensuales de las variables macroeconómicas
# Asegurar que las columnas sean numéricas
for col in ['ipc', 'pib', 'tasa_credito', 'salario_minimo']:
    df_final[col] = pd.to_numeric(df_final[col], errors='coerce')

# Ordenar por fecha para asegurar el cálculo correcto de las variaciones
df_final = df_final.sort_values('fecha_cierre')

# Calcular las variaciones mensuales (respecto al mes anterior)
df_final['var_ipc'] = df_final['ipc'].pct_change(periods=1) * 100
df_final['var_pib'] = df_final['pib'].pct_change(periods=1) * 100
df_final['var_tasa_credito'] = df_final['tasa_credito'].pct_change(periods=1) * 100
df_final['var_salario_minimo'] = df_final['salario_minimo'].pct_change(periods=1) * 100

# Volver a ordenar por cliente y fecha para mantener la consistencia
df_final = df_final.sort_values(['id_cliente', 'fecha_cierre'])

In [43]:
# Verificar las variables macroeconómicas y sus variaciones mensuales
print("Resumen de variables macroeconómicas:")
columnas_macro = ['ipc', 'pib', 'tasa_credito', 'salario_minimo', 
                  'var_ipc', 'var_pib', 'var_tasa_credito', 'var_salario_minimo']

# Mostrar estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df_final[columnas_macro].describe())

# Mostrar algunos valores ejemplo ordenados por fecha
print("\nEjemplos de valores y variaciones mensuales (ordenados por fecha):")
print("\nNota: Las variaciones son respecto al mes anterior")
df_ejemplo = df_final[['fecha_cierre'] + columnas_macro].sort_values('fecha_cierre').head(24)
print(df_ejemplo.to_string())

Resumen de variables macroeconómicas:

Estadísticas descriptivas:
       ipc  pib  tasa_credito  salario_minimo  var_ipc  var_pib  \
count  0.0  0.0           0.0             0.0      0.0      0.0   
mean   NaN  NaN           NaN             NaN      NaN      NaN   
std    NaN  NaN           NaN             NaN      NaN      NaN   
min    NaN  NaN           NaN             NaN      NaN      NaN   
25%    NaN  NaN           NaN             NaN      NaN      NaN   
50%    NaN  NaN           NaN             NaN      NaN      NaN   
75%    NaN  NaN           NaN             NaN      NaN      NaN   
max    NaN  NaN           NaN             NaN      NaN      NaN   

       var_tasa_credito  var_salario_minimo  
count               0.0                 0.0  
mean                NaN                 NaN  
std                 NaN                 NaN  
min                 NaN                 NaN  
25%                 NaN                 NaN  
50%                 NaN                 NaN  
75%     

## Definición de la variable objetivo (Default)
Crearemos la variable objetivo basada en dos criterios:
1. Días de mora > 90 días
2. Calificación crediticia (D o E)

In [44]:
# Definir default basado en días de mora y calificación
df_final['default'] = ((df_final['dias_mora'] > 90) | 
                      (df_final['calificacion'].isin(['D', 'E']))).astype(int)

# Calcular la tasa de default
tasa_default = df_final['default'].mean() * 100

print(f"Tasa de default en la cartera: {tasa_default:.2f}%")

# Mostrar distribución de defaults por categoría de riesgo
print("\nDistribución de defaults por categoría de riesgo:")
print(pd.crosstab(df_final['categoria_riesgo'], df_final['default'], 
                  normalize='index') * 100)

# Guardar el DataFrame final procesado
df_final.to_csv('../data/processed/datos_modelado.csv', index=False)
print("\nDataset guardado en '../data/processed/datos_modelado.csv'")

Tasa de default en la cartera: 12.32%

Distribución de defaults por categoría de riesgo:
default                            0          1
categoria_riesgo                               
A - Riesgo Bajo           100.000000   0.000000
B - Riesgo Moderado Bajo  100.000000   0.000000
C - Riesgo Moderado       100.000000   0.000000
D - Riesgo Moderado Alto   98.206495   1.793505
E - Riesgo Alto            85.937843  14.062157

Dataset guardado en '../data/processed/datos_modelado.csv'

Dataset guardado en '../data/processed/datos_modelado.csv'


In [45]:
df_final

,fecha_cierre,id_cliente,dias_mora,calificacion,cuenta_cobrar,deterioro,id_negocio,cod_negocio,id_linea,nombre_linea,...,categoria_riesgo,ipc,pib,tasa_credito,salario_minimo,default,var_ipc,var_pib,var_tasa_credito,var_salario_minimo
8476,2024-04-30,023c07084a0d5940f2de0e11ff05fbc8f2077d085144c0...,0,A,7.735000e+05,1.701700e+04,77c9010f2baaf521cdd124557fc731fcabee2ddd240fd5...,77c9010f2baaf521cdd124557fc731fcabee2ddd240fd5...,2.0,Inmobiliario,...,A - Riesgo Bajo,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
9049,2024-05-31,023c07084a0d5940f2de0e11ff05fbc8f2077d085144c0...,0,A,0.000000e+00,0.000000e+00,77c9010f2baaf521cdd124557fc731fcabee2ddd240fd5...,77c9010f2baaf521cdd124557fc731fcabee2ddd240fd5...,2.0,Inmobiliario,...,A - Riesgo Bajo,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
3588,2023-07-31,0265ed262c8b358431e8233e6de623825fb5827ac49a90...,0,A,6.429008e+09,1.414382e+09,0265ed262c8b358431e8233e6de623825fb5827ac49a90...,0265ed262c8b358431e8233e6de623825fb5827ac49a90...,3.0,Administración,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4258,2023-08-31,027746ade127f44a65f74b3a169c981b097da9e125d513...,0,A,6.902000e+05,1.518440e+05,423bfa8170845487fc09a3d2111e50b6f30b517dd998d2...,423bfa8170845487fc09a3d2111e50b6f30b517dd998d2...,2.0,Inmobiliario,...,D - Riesgo Moderado Alto,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4883,2023-09-30,027746ade127f44a65f74b3a169c981b097da9e125d513...,0,A,0.000000e+00,0.000000e+00,423bfa8170845487fc09a3d2111e50b6f30b517dd998d2...,423bfa8170845487fc09a3d2111e50b6f30b517dd998d2...,2.0,Inmobiliario,...,D - Riesgo Moderado Alto,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12274,2024-11-30,ff77d1fd01ee1f5d70c79693d6ac9bf781d0f4cce978e8...,26,A,9.282000e+06,2.042040e+05,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,99.0,Otros,...,E - Riesgo Alto,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
12770,2024-12-31,ff77d1fd01ee1f5d70c79693d6ac9bf781d0f4cce978e8...,57,B,1.392300e+07,5.290740e+05,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,99.0,Otros,...,E - Riesgo Alto,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
13109,2025-01-31,ff77d1fd01ee1f5d70c79693d6ac9bf781d0f4cce978e8...,0,A,0.000000e+00,0.000000e+00,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,99.0,Otros,...,E - Riesgo Alto,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
11028,2024-09-30,ff80bb5730abd30c3495ddc72d3f710674294a348a486f...,0,A,1.190000e+07,2.618000e+04,5e5faf72f089c72f90e5ec4b3896b95545495ec7cfd9e3...,5e5faf72f089c72f90e5ec4b3896b95545495ec7cfd9e3...,4.0,Garantía,...,A - Riesgo Bajo,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
